# Video CLassifier

## Machine Learning Model

In [1]:
import os
import time
import torch
import random
import torchvision.transforms as T
import torchvision.models as models
import torchvision.io as io
from torch.utils.data import Dataset, DataLoader, Subset
from torch import nn, optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import numpy as np

torch._dynamo.config.suppress_errors = True
torch._dynamo.disable()
# ✅ CONFIG
AUTISM_DIR = "Autism"
NO_AUTISM_DIR = "No_Autism"
NUM_FRAMES = 16
IMG_SIZE = 224
EPOCHS = 10
BATCH_SIZE = 2
FOLDS = 10
DEVICE = torch.device("cpu")

# ✅ Collect video paths and labels
video_paths, labels = [], []
for f in os.listdir(AUTISM_DIR):
    if f.endswith(".mp4"):
        video_paths.append(os.path.join(AUTISM_DIR, f))
        labels.append(1)
for f in os.listdir(NO_AUTISM_DIR):
    if f.endswith(".mp4"):
        video_paths.append(os.path.join(NO_AUTISM_DIR, f))
        labels.append(0)

# ✅ Video Dataset
class VideoFrameDataset(Dataset):
    def __init__(self, paths, labels, num_frames=NUM_FRAMES, transform=None):
        self.paths = paths
        self.labels = labels
        self.num_frames = num_frames
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        video_path = self.paths[idx]
        label = torch.tensor([self.labels[idx]], dtype=torch.float32)

        frames, _, _ = io.read_video(video_path, pts_unit='sec')
        total = frames.shape[0]
        indices = torch.linspace(0, total - 1, steps=self.num_frames).long()
        sampled_frames = frames[indices]

        processed = []
        for frame in sampled_frames:
            frame = frame.permute(2, 0, 1).float() / 255.0  # CxHxW
            if self.transform:
                frame = self.transform(frame)
            processed.append(frame)

        return torch.stack(processed), label

# ✅ Transform
transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
])

# ✅ Model: ResNet18 (pretrained) + BiLSTM + Dropout
class ResNet18BiLSTM(nn.Module):
    def __init__(self, hidden_dim=128, num_layers=1, dropout_p=0.3):
        super().__init__()
        base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        base_model.fc = nn.Identity()
        self.cnn = base_model

        self.lstm = nn.LSTM(
            input_size=512,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        self.dropout = nn.Dropout(dropout_p)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x):  # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        feats = self.cnn(x)
        feats = feats.view(B, T, -1)
        _, (hn, _) = self.lstm(feats)
        hn = torch.cat((hn[-2], hn[-1]), dim=1)  # bidirectional output
        return self.fc(self.dropout(hn))  # (B, 1)

# ✅ 10-Fold Cross Validation
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
dataset = VideoFrameDataset(video_paths, labels, transform=transform)

total_start = time.time()

for fold, (train_idx, test_idx) in enumerate(kf.split(video_paths, labels)):
    print(f"\n===== Fold {fold+1}/{FOLDS} =====")
    fold_start = time.time()

    train_loader = DataLoader(Subset(dataset, train_idx), batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(Subset(dataset, test_idx), batch_size=BATCH_SIZE)

    model = ResNet18BiLSTM().to(DEVICE)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(EPOCHS):
        epoch_start = time.time()
        model.train()
        total_loss, correct, total = 0, 0, 0
        print(f"\nEpoch {epoch+1}/{EPOCHS}")

        for i, (videos, labels_batch) in enumerate(train_loader):
            videos, labels_batch = videos.to(DEVICE), labels_batch.to(DEVICE)
            optimizer.zero_grad()
            logits = model(videos).squeeze(1)
            loss = criterion(logits, labels_batch.squeeze(1))
            loss.backward()
            optimizer.step()

            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).float()
            correct += (preds == labels_batch.squeeze(1)).sum().item()
            total += labels_batch.size(0)
            total_loss += loss.item()

            print(f"  Batch {i+1}/{len(train_loader)} - Loss: {loss.item():.4f} - Acc: {(preds == labels_batch.squeeze(1)).float().mean():.2f}")

        print(f"  Epoch Accuracy: {correct / total:.4f} | Avg Loss: {total_loss / len(train_loader):.4f}")
        epoch_end = time.time()
        print(f"⏱️  Epoch Time: {epoch_end - epoch_start:.2f} seconds")

    # ✅ Evaluation
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for videos, labels_batch in test_loader:
            videos, labels_batch = videos.to(DEVICE), labels_batch.to(DEVICE)
            logits = model(videos).squeeze(1)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).float()

            correct += (preds == labels_batch.squeeze(1)).sum().item()
            total += labels_batch.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())

    print(f"\n✅ Test Accuracy for Fold {fold+1}: {100 * correct / total:.2f}%")
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["No Autism", "Autism"]))

    fold_end = time.time()
    print(f"🕒 Fold {fold+1} Time: {fold_end - fold_start:.2f} seconds")

total_end = time.time()
print(f"\n⏳ Total Training Time: {total_end - total_start:.2f} seconds")


ModuleNotFoundError: No module named 'torchvision'

## Metrics Calculation

In [7]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Load the CSV
df = pd.read_csv('videoresults.csv')

# Replace with your actual column names if different
y_true = df['TrueLabel']
y_pred = df['PredictedLabel']

# Compute metrics
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')
accuracy = accuracy_score(y_true, y_pred)

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Accuracy:  {accuracy:.4f}")


Precision: 0.9038
Recall:    0.9400
F1 Score:  0.9216
Accuracy:  0.9200


# Avatar Classifier

## Remove Extraneuous Data

In [15]:
# ✅ FBX → NPY extraction using Blender (Jupyter-compatible wrapper)
def extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script):
    def call_blender(input_fbx, output_npy):
        cmd = [blender_exec, "--background", "--python", blender_script, "--", input_fbx, output_npy]
        result = subprocess.run(cmd, capture_output=True, text=True)
        print(result.stdout)
        if result.stderr:
            print("⚠️ Blender Error:", result.stderr)

    os.makedirs(fbx_root, exist_ok=True)
    os.makedirs(npy_root, exist_ok=True)
    for label in ["Autism", "No_Autism"]:
        fbx_class_path = os.path.join(fbx_root, label)
        npy_class_path = os.path.join(npy_root, label)
        os.makedirs(fbx_class_path, exist_ok=True)
        os.makedirs(npy_class_path, exist_ok=True)

    all_fbx_files = []
    for label in os.listdir(fbx_root):
        label_dir = os.path.join(fbx_root, label)
        if not os.path.isdir(label_dir): continue
        out_dir = os.path.join(npy_root, label)
        os.makedirs(out_dir, exist_ok=True)
        for fbx_file in os.listdir(label_dir):
            if not fbx_file.endswith(".fbx"): continue
            input_fbx = os.path.join(label_dir, fbx_file)
            output_npy = os.path.join(out_dir, fbx_file.replace(".fbx", ".npy"))
            all_fbx_files.append((input_fbx, output_npy))

    print(f"\n🟡 Extracting {len(all_fbx_files)} files with Blender...")
    for input_fbx, output_npy in tqdm(all_fbx_files, desc="Extracting", unit="file"):
        call_blender(input_fbx, output_npy)

# ✅ Dataset Wrapper
class SkeletonDataset(Dataset):
    def __init__(self, npy_paths, labels):
        self.paths = npy_paths
        self.labels = labels

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        data = np.load(self.paths[idx])  # [T, J, 3]
        x = torch.tensor(data, dtype=torch.float32).permute(2, 0, 1)  # [C, T, V]
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y, self.paths[idx]

fbx_root = "Avatar Dataset"
npy_root = os.path.join(os.getcwd(), "npy_data")
blender_exec = "/Applications/Blender.app/Contents/MacOS/Blender"
blender_script = 'fbx_to_npy.py'

extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script)


🟡 Extracting 200 files with Blender...


Extracting:   0%|▏                            | 1/200 [00:05<19:43,  5.95s/file]

FBX version: 7700
Error, could not find any file path in  FBXElem(id=b'Texture', props=[3147589601440, b'Map #1\x00\x01Texture', b''], props_type=bytearray(b'LSS'), elems=[FBXElem(id=b'Type', props=[b'TextureVideoClip'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Version', props=[202], props_type=bytearray(b'I'), elems=[]), FBXElem(id=b'TextureName', props=[b'Map #1\x00\x01Texture'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Properties70', props=[], props_type=bytearray(b''), elems=[FBXElem(id=b'P', props=[b'3dsMax', b'Compound', b'', b''], props_type=bytearray(b'SSSS'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDa', b'int', b'Integer', b'', 592], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDb', b'int', b'Integer', b'', 0], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|SuperClassID', b'int', b'Integer', b'', 3088], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|pa

Extracting:   1%|▎                            | 2/200 [00:07<11:50,  3.59s/file]

FBX version: 7700
Error, could not find any file path in  FBXElem(id=b'Texture', props=[2937412834816, b'Map #1\x00\x01Texture', b''], props_type=bytearray(b'LSS'), elems=[FBXElem(id=b'Type', props=[b'TextureVideoClip'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Version', props=[202], props_type=bytearray(b'I'), elems=[]), FBXElem(id=b'TextureName', props=[b'Map #1\x00\x01Texture'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Properties70', props=[], props_type=bytearray(b''), elems=[FBXElem(id=b'P', props=[b'3dsMax', b'Compound', b'', b''], props_type=bytearray(b'SSSS'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDa', b'int', b'Integer', b'', 592], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDb', b'int', b'Integer', b'', 0], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|SuperClassID', b'int', b'Integer', b'', 3088], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|pa

...

FBX version: 7700
Error, could not find any file path in  FBXElem(id=b'Texture', props=[2553769110672, b'Map #1\x00\x01Texture', b''], props_type=bytearray(b'LSS'), elems=[FBXElem(id=b'Type', props=[b'TextureVideoClip'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Version', props=[202], props_type=bytearray(b'I'), elems=[]), FBXElem(id=b'TextureName', props=[b'Map #1\x00\x01Texture'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Properties70', props=[], props_type=bytearray(b''), elems=[FBXElem(id=b'P', props=[b'3dsMax', b'Compound', b'', b''], props_type=bytearray(b'SSSS'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDa', b'int', b'Integer', b'', 592], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDb', b'int', b'Integer', b'', 0], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|SuperClassID', b'int', b'Integer', b'', 3088], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|pa

Extracting: 100%|██████████████████████████▊| 199/200 [06:24<00:01,  1.96s/file]

FBX version: 7700
Error, could not find any file path in  FBXElem(id=b'Texture', props=[2527448906864, b'Map #1\x00\x01Texture', b''], props_type=bytearray(b'LSS'), elems=[FBXElem(id=b'Type', props=[b'TextureVideoClip'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Version', props=[202], props_type=bytearray(b'I'), elems=[]), FBXElem(id=b'TextureName', props=[b'Map #1\x00\x01Texture'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Properties70', props=[], props_type=bytearray(b''), elems=[FBXElem(id=b'P', props=[b'3dsMax', b'Compound', b'', b''], props_type=bytearray(b'SSSS'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDa', b'int', b'Integer', b'', 592], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDb', b'int', b'Integer', b'', 0], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|SuperClassID', b'int', b'Integer', b'', 3088], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|pa

Extracting: 100%|███████████████████████████| 200/200 [06:26<00:00,  1.93s/file]

FBX version: 7700
Error, could not find any file path in  FBXElem(id=b'Texture', props=[2052986238256, b'Map #1\x00\x01Texture', b''], props_type=bytearray(b'LSS'), elems=[FBXElem(id=b'Type', props=[b'TextureVideoClip'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Version', props=[202], props_type=bytearray(b'I'), elems=[]), FBXElem(id=b'TextureName', props=[b'Map #1\x00\x01Texture'], props_type=bytearray(b'S'), elems=[]), FBXElem(id=b'Properties70', props=[], props_type=bytearray(b''), elems=[FBXElem(id=b'P', props=[b'3dsMax', b'Compound', b'', b''], props_type=bytearray(b'SSSS'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDa', b'int', b'Integer', b'', 592], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|ClassIDb', b'int', b'Integer', b'', 0], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|SuperClassID', b'int', b'Integer', b'', 3088], props_type=bytearray(b'SSSSI'), elems=[]), FBXElem(id=b'P', props=[b'3dsMax|pa

## Models Used

In [17]:
# ✅ FBX to ST-GCN Classifier Pipeline (With Upgraded Architecture)
import os
import numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import csv
import time
from tqdm import trange

# ✅ Residual Block with Temporal Kernel, BatchNorm, and Dropout
class STGCNResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(9,1), stride=(1,1), dropout=0.3):
        super().__init__()
        padding = ((kernel_size[0] - 1) // 2, 0)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) if dropout else nn.Identity()
        self.res_conv = (
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            if in_channels != out_channels or stride != (1,1)
            else nn.Identity()
        )

    def forward(self, x):
        res = self.res_conv(x)
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.dropout(out)
        return self.relu(out + res)

# ✅ Full ST-GCN Model with Residual Blocks and Optional LSTM
class STGCNModel(nn.Module):
    def __init__(self, in_channels=3, num_joints=250, num_classes=2, use_lstm=True):
        super().__init__()
        self.A = torch.eye(num_joints)
        self.block1 = STGCNResidualBlock(in_channels, 64, dropout=0.1)
        self.pool1 = nn.MaxPool2d((2,1))
        self.block2 = STGCNResidualBlock(64, 128, dropout=0.1)
        self.pool2 = nn.MaxPool2d((2,1))
        self.block3 = STGCNResidualBlock(128, 256, dropout=0.1)
        self.use_lstm = use_lstm

        if self.use_lstm:
            self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
            self.fc = nn.Linear(128, num_classes)
        else:
            self.fc = nn.Linear(256, num_classes)

    def forward(self, x):  # x: [B, C, T, V]
        A = self.A.to(x.device)
        x = torch.einsum('nctv,vw->nctw', x, A)
        x = self.block1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = self.block3(x)
        x = x.mean(3)  # average over joints: [B, 256, T]

        if self.use_lstm:
            x = x.permute(0, 2, 1)  # [B, T, 256]
            out, _ = self.lstm(x)
            x = out[:, -1, :]
        else:
            x = x.mean(2)  # average over time

        return self.fc(x)

# ✅ Dataset Class
class SkeletonDataset(Dataset):
    def __init__(self, file_paths, labels, augment=False):
        self.file_paths = file_paths
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        data = np.load(self.file_paths[idx])
        if data.ndim == 3:
            data = np.transpose(data, (1, 2, 0))
            data = np.transpose(data, (2, 0, 1))
        x = torch.tensor(data, dtype=torch.float32)
        if self.augment:
            x += torch.randn_like(x) * 0.005
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y, self.file_paths[idx]

# ✅ Training Loop

def train_stgcn(npy_dir, label_map, pretrained_ckpt=None, epochs=75, folds=10, output_csv='resultsOne.csv'):
    all_paths, all_labels = [], []
    for label, class_id in label_map.items():
        class_dir = os.path.join(npy_dir, label)
        if not os.path.isdir(class_dir):
            print(f"⚠️ Skipping missing folder: {class_dir}")
            continue
        for f in os.listdir(class_dir):
            if f.endswith(".npy"):
                all_paths.append(os.path.join(class_dir, f))
                all_labels.append(class_id)

    if len(all_paths) == 0:
        print("❌ No .npy data found. Aborting training.")
        return

    paths = np.array(all_paths)
    labels = np.array(all_labels)
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    csv_rows = [["Fold", "VideoPath", "PredictedLabel", "TrueLabel"]]

    for fold, (train_idx, test_idx) in enumerate(skf.split(paths, labels)):
        print(f"\n=== Fold {fold+1}/{folds} ===")
        train_ds = SkeletonDataset(paths[train_idx], labels[train_idx], augment=True)
        test_ds  = SkeletonDataset(paths[test_idx], labels[test_idx], augment=False)
        train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
        test_dl  = DataLoader(test_ds, batch_size=4)

        C, T, V = train_ds[0][0].shape
        model = STGCNModel(in_channels=C, num_joints=V, num_classes=2).to("cpu")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        from collections import Counter
        label_counts = Counter(labels)
        total = sum(label_counts.values())
        weights = [total / label_counts[i] for i in range(len(label_map))]
        weights = torch.tensor(weights, dtype=torch.float32)
        criterion = nn.CrossEntropyLoss(weight=weights)

        best_val_acc = 0.0
        epochs_no_improve = 0
        patience = 10

        warmup_epochs = 10
        for epoch in trange(epochs, desc=f"Fold {fold+1} Training", unit="epoch"):
            model.train()
            losses = []
            for x, y, _ in train_dl:
                x, y = x.to("cpu"), y.to("cpu")
                pred = model(x)
                loss = criterion(pred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
            print(f"  Epoch {epoch+1} Loss: {np.mean(losses):.4f}")
            scheduler.step()

            model.eval()
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for x, y, _ in test_dl:
                    out = model(x)
                    preds = torch.argmax(out, dim=1)
                    val_correct += (preds == y).sum().item()
                    val_total += y.size(0)
            val_acc = val_correct / val_total
            print(f"  Validation Accuracy: {val_acc:.4f}")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                epochs_no_improve = 0
                best_model_path = f"best_model_fold{fold+1}.pth"
                torch.save(model.state_dict(), best_model_path)
            elif epoch >= warmup_epochs:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"⏹️ Early stopping on fold {fold+1} at epoch {epoch+1}")
                    break

        model.load_state_dict(torch.load(f"best_model_fold{fold+1}.pth"))
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for x, y, paths_batch in test_dl:
                out = model(x)
                preds = torch.argmax(out, dim=1)
                y_true.extend(y.tolist())
                y_pred.extend(preds.tolist())
                for i in range(len(paths_batch)):
                    csv_rows.append([fold + 1, paths_batch[i], int(preds[i].item()), int(y[i].item())])

        print(classification_report(y_true, y_pred, target_names=label_map.keys(), zero_division=0))
        print("Confusion Matrix:")
        print(confusion_matrix(y_true, y_pred))
        print("F1 Score (per class):")
        print(f1_score(y_true, y_pred, average=None))

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(csv_rows)
    print(f"\n✅ Results saved to: {output_csv}")

# ✅ FBX Extraction Logic (Assumed Already Defined)
def extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script):
    def call_blender(input_fbx, output_npy):
        cmd = [blender_exec, "--background", "--python", blender_script, "--", input_fbx, output_npy]
        subprocess.run(cmd, capture_output=True)

    os.makedirs(npy_root, exist_ok=True)
    for label in ["Autism", "No_Autism"]:
        class_dir = os.path.join(fbx_root, label)
        out_dir = os.path.join(npy_root, label)
        os.makedirs(out_dir, exist_ok=True)
        for fbx_file in os.listdir(class_dir):
            if fbx_file.endswith(".fbx"):
                input_fbx = os.path.join(class_dir, fbx_file)
                output_npy = os.path.join(out_dir, fbx_file.replace(".fbx", ".npy"))
                call_blender(input_fbx, output_npy)

# ✅ Run
npy_root = os.path.join(os.getcwd(), "npy_data")
pretrained_ckpt = None
label_map = {"No_Autism": 0, "Autism": 1}

train_stgcn(npy_dir=npy_root, label_map=label_map, pretrained_ckpt=pretrained_ckpt)



=== Fold 1/10 ===


Fold 1 Training:   1%|▎                       | 1/75 [00:01<01:22,  1.12s/epoch]

  Epoch 1 Loss: 0.7309
  Validation Accuracy: 0.5000


Fold 1 Training:   3%|▋                       | 2/75 [00:02<01:22,  1.14s/epoch]

  Epoch 2 Loss: 0.6997
  Validation Accuracy: 0.5000


KeyboardInterrupt: 

In [18]:
# ✅ FBX to ST-GCN Classifier Pipeline (Enhanced with Velocity + Bone + Temporal Convs)
import os
import numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import csv
from tqdm import trange

# ✅ Residual Block with Wider Temporal Kernel
class STGCNResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(9,1), stride=(1,1), dropout=0.3):
        super().__init__()
        padding = ((kernel_size[0] - 1) // 2, 0)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) if dropout else nn.Identity()
        self.res_conv = (
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            if in_channels != out_channels or stride != (1,1)
            else nn.Identity()
        )

    def forward(self, x):
        res = self.res_conv(x)
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.dropout(out)
        return self.relu(out + res)

# ✅ Full ST-GCN Model with Learnable Graph, Residuals, Temporal Conv, LSTM, Attention, and Positional Encoding
class STGCNModel(nn.Module):
    def __init__(self, in_channels=9, num_joints=250, num_classes=2, use_lstm=True):
        super().__init__()
        self.A_raw = nn.Parameter(torch.randn(num_joints, num_joints) * 0.1)
        self.A_activation = nn.Softmax(dim=-1)
        self.block1 = STGCNResidualBlock(in_channels, 64, kernel_size=(9,1), dropout=0.1)
        self.pool1 = nn.MaxPool2d((2,1))
        self.block2 = STGCNResidualBlock(64, 128, kernel_size=(9,1), dropout=0.1)
        self.pool2 = nn.AdaptiveMaxPool2d((1, 1))
        self.block3 = STGCNResidualBlock(128, 256, kernel_size=(9,1), dropout=0.1)
        self.use_lstm = use_lstm

        if self.use_lstm:
            self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
            self.attn = nn.MultiheadAttention(embed_dim=128, num_heads=4, batch_first=True)
            self.fc = nn.Linear(128, num_classes)
        else:
            self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        A = self.A_activation(self.A_raw.to(x.device))
        x = torch.einsum('nctv,vw->nctw', x, A)
        x = self.block1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = self.block3(x)
        x = x.mean(3)

        # ✅ Positional Encoding
        pos_encoding = torch.arange(x.shape[1], device=x.device).float().unsqueeze(0).unsqueeze(2)
        x = x + pos_encoding  # [B, T, C] implicit broadcast

        if self.use_lstm:
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            x, _ = self.attn(out, out, out)
            x = x.mean(dim=1)
        else:
            x = x.mean(2)

        return self.fc(x)

# ✅ Skeleton Dataset with Velocity + Bone Feature Augmentation
class SkeletonDataset(Dataset):
    def __init__(self, file_paths, labels, augment=False):
        self.file_paths = file_paths
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        data = np.load(self.file_paths[idx])
        if data.ndim == 3:
            data = np.transpose(data, (1, 2, 0))

        joints = torch.tensor(data, dtype=torch.float32)
        velocity = joints[:, 1:, :] - joints[:, :-1, :]
        velocity = F.pad(velocity, (0, 0, 0, 1), mode='replicate')

        bone_pairs = [(i, i + 1) for i in range(joints.shape[2] - 1)]
        bone_vectors = [joints[:, :, i] - joints[:, :, j] for i, j in bone_pairs]
        bones = torch.stack(bone_vectors, dim=2)
        bones = F.pad(bones, (0, joints.shape[2] - bones.shape[2]))

        x = torch.cat([joints, velocity, bones], dim=0)

        if self.augment:
            x += torch.randn_like(x) * 0.005

        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y, self.file_paths[idx]

# ✅ Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.alpha, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

# ✅ Training Loop (continued)
def train_stgcn(npy_dir, label_map, pretrained_ckpt=None, epochs=75, folds=10, output_csv='ResultsTwo.csv'):
    # 📍 Results CSV will be saved here:
    # → /Users/chancellorwoolsey/Documents/Research Work/images_and_videos/Avatar Dataset/results.csv

    fold_metrics = []
    all_paths, all_labels = [], []
    for label, class_id in label_map.items():
        class_dir = os.path.join(npy_dir, label)
        if not os.path.isdir(class_dir):
            continue
        for f in os.listdir(class_dir):
            if f.endswith(".npy"):
                all_paths.append(os.path.join(class_dir, f))
                all_labels.append(class_id)

    paths = np.array(all_paths)
    labels = np.array(all_labels)
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    csv_rows = [["Fold", "VideoPath", "PredictedLabel", "TrueLabel"]]

    for fold, (train_idx, test_idx) in enumerate(skf.split(paths, labels)):
        train_ds = SkeletonDataset(paths[train_idx], labels[train_idx], augment=True)
        test_ds = SkeletonDataset(paths[test_idx], labels[test_idx], augment=False)
        train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
        test_dl = DataLoader(test_ds, batch_size=4)

        C, T, V = train_ds[0][0].shape
        model = STGCNModel(in_channels=C, num_joints=V, num_classes=2).to("cpu")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        label_weights = torch.tensor([1.0 for _ in label_map], dtype=torch.float32)
        criterion = FocalLoss(alpha=label_weights)

        best_val_acc = 0.0
        patience, warmup = 10, 10
        epochs_no_improve = 0

        for epoch in trange(epochs, desc=f"Fold {fold+1} Training"):
            model.train()
            for x, y, _ in train_dl:
                x, y = x.to("cpu"), y.to("cpu")
                loss = criterion(model(x), y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            scheduler.step()

            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for x, y, _ in test_dl:
                    preds = torch.argmax(model(x), dim=1)
                    val_correct += (preds == y).sum().item()
                    val_total += y.size(0)
            val_acc = val_correct / val_total
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")
                epochs_no_improve = 0
            elif epoch >= warmup:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    break

        model.load_state_dict(torch.load(f"best_model_fold{fold+1}.pth"))
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for x, y, paths_batch in test_dl:
                out = model(x)
                preds = torch.argmax(out, dim=1)
                y_true.extend(y.tolist())
                y_pred.extend(preds.tolist())
                for i in range(len(paths_batch)):
                    csv_rows.append([fold + 1, paths_batch[i], int(preds[i].item()), int(y[i].item())])

        acc = np.mean(np.array(y_true) == np.array(y_pred))
        f1 = f1_score(y_true, y_pred, average='macro')
        prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
        fold_metrics.append({"Fold": fold+1, "Accuracy": acc, "F1": f1, "Precision": prec, "Recall": rec})

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(csv_rows)
    print(f"Results saved to: {os.path.abspath(output_csv)}")

    metrics_df = pd.DataFrame(fold_metrics)
    mean_row = metrics_df.mean(numeric_only=True)
    mean_row["Fold"] = "Average"
    metrics_df = pd.concat([metrics_df, pd.DataFrame([mean_row])], ignore_index=True)
    summary_path = output_csv.replace(".csv", "_metrics_summary.csv")
    metrics_df.to_csv(summary_path, index=False)
    print(f"Summary saved to: {os.path.abspath(summary_path)}")

# ✅ FBX Extraction Logic

def extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script):
    def call_blender(input_fbx, output_npy):
        cmd = [blender_exec, "--background", "--python", blender_script, "--", input_fbx, output_npy]
        subprocess.run(cmd, capture_output=True)

    os.makedirs(npy_root, exist_ok=True)
    for label in ["Autism", "No_Autism"]:
        class_dir = os.path.join(fbx_root, label)
        out_dir = os.path.join(npy_root, label)
        os.makedirs(out_dir, exist_ok=True)
        for fbx_file in os.listdir(class_dir):
            if fbx_file.endswith(".fbx"):
                input_fbx = os.path.join(class_dir, fbx_file)
                output_npy = os.path.join(out_dir, fbx_file.replace(".fbx", ".npy"))
                call_blender(input_fbx, output_npy)

# ✅ Execution Entrypoint
if __name__ == "__main__":
    npy_root = os.path.join(os.getcwd(), "npy_data")
    label_map = {"No_Autism": 0, "Autism": 1}
    train_stgcn(npy_dir=npy_root, label_map=label_map)


Fold 1 Training:   3%|▋                          | 2/75 [00:05<03:10,  2.62s/it]


KeyboardInterrupt: 

In [20]:
# ✅ FBX to ST-GCN Classifier Pipeline (Original Stable Version)
import os
import numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import csv
from tqdm import trange

# ✅ Residual Block with Wider Temporal Kernel
class STGCNResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(9,1), stride=(1,1), dropout=0.3):
        super().__init__()
        padding = ((kernel_size[0] - 1) // 2, 0)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) if dropout else nn.Identity()
        self.res_conv = (
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            if in_channels != out_channels or stride != (1,1)
            else nn.Identity()
        )

    def forward(self, x):
        res = self.res_conv(x)
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.dropout(out)
        return self.relu(out + res)

# ✅ Full ST-GCN Model (Stable: no edge weights, no learnable A, no attention)
class STGCNModel(nn.Module):
    def __init__(self, in_channels=6, num_joints=250, num_classes=2, use_lstm=True):
        super().__init__()
        self.A = torch.eye(num_joints)
        self.block1 = STGCNResidualBlock(in_channels, 64, dropout=0.1)
        self.pool1 = nn.MaxPool2d((2,1))
        self.block2 = STGCNResidualBlock(64, 128, dropout=0.1)
        self.pool2 = nn.AdaptiveMaxPool2d((1, 1))
        self.block3 = STGCNResidualBlock(128, 256, dropout=0.1)
        self.use_lstm = use_lstm

        if self.use_lstm:
            self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
            self.fc = nn.Linear(128, num_classes)
        else:
            self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        A = self.A.to(x.device)
        x = torch.einsum('nctv,vw->nctw', x, A)
        x = self.block1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = self.block3(x)
        x = x.mean(3)

        if self.use_lstm:
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            x = out[:, -1, :]
        else:
            x = x.mean(2)

        return self.fc(x)

# ✅ Dataset with Velocity Only (Stable)
class SkeletonDataset(Dataset):
    def __init__(self, file_paths, labels, augment=False):
        self.file_paths = file_paths
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        data = np.load(self.file_paths[idx])
        if data.ndim == 3:
            data = np.transpose(data, (1, 2, 0))

        joints = torch.tensor(data, dtype=torch.float32)
        velocity = joints[:, 1:, :] - joints[:, :-1, :]
        velocity = F.pad(velocity, (0, 0, 0, 1), mode='replicate')
        x = torch.cat([joints, velocity], dim=0)

        if self.augment:
            x += torch.randn_like(x) * 0.005

        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y, self.file_paths[idx]

# ✅ Output files will be saved here:
#     → Results saved to: /Users/chancellorwoolsey/Documents/Research Work/images_and_videos/Avatar Dataset

def train_stgcn(npy_dir, label_map, pretrained_ckpt=None, epochs=75, folds=10, output_csv='ResultsThree.csv'):
    fold_metrics = []
    all_paths, all_labels = [], []
    for label, class_id in label_map.items():
        class_dir = os.path.join(npy_dir, label)
        if not os.path.isdir(class_dir):
            continue
        for f in os.listdir(class_dir):
            if f.endswith(".npy"):
                all_paths.append(os.path.join(class_dir, f))
                all_labels.append(class_id)

    paths = np.array(all_paths)
    labels = np.array(all_labels)
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    csv_rows = [["Fold", "VideoPath", "PredictedLabel", "TrueLabel"]]

    for fold, (train_idx, test_idx) in enumerate(skf.split(paths, labels)):
        train_ds = SkeletonDataset(paths[train_idx], labels[train_idx], augment=True)
        test_ds = SkeletonDataset(paths[test_idx], labels[test_idx], augment=False)
        train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
        test_dl = DataLoader(test_ds, batch_size=4)

        C, T, V = train_ds[0][0].shape
        model = STGCNModel(in_channels=C, num_joints=V, num_classes=2).to("cpu")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        label_weights = torch.tensor([1.0 for _ in label_map], dtype=torch.float32)
        criterion = nn.CrossEntropyLoss(weight=label_weights)

        best_val_acc = 0.0
        patience, warmup = 10, 10
        epochs_no_improve = 0

        for epoch in trange(epochs, desc=f"Fold {fold+1} Training"):
            model.train()
            for x, y, _ in train_dl:
                x, y = x.to("cpu"), y.to("cpu")
                loss = criterion(model(x), y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            scheduler.step()

            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for x, y, _ in test_dl:
                    preds = torch.argmax(model(x), dim=1)
                    val_correct += (preds == y).sum().item()
                    val_total += y.size(0)
            val_acc = val_correct / val_total
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")
                epochs_no_improve = 0
            elif epoch >= warmup:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    break

        model.load_state_dict(torch.load(f"best_model_fold{fold+1}.pth"))
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for x, y, paths_batch in test_dl:
                out = model(x)
                preds = torch.argmax(out, dim=1)
                y_true.extend(y.tolist())
                y_pred.extend(preds.tolist())
                for i in range(len(paths_batch)):
                    csv_rows.append([fold + 1, paths_batch[i], int(preds[i].item()), int(y[i].item())])

        acc = np.mean(np.array(y_true) == np.array(y_pred))
        f1 = f1_score(y_true, y_pred, average='macro')
        prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
        fold_metrics.append({"Fold": fold+1, "Accuracy": acc, "F1": f1, "Precision": prec, "Recall": rec})

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(csv_rows)
    print(f"Results saved to: {os.path.abspath(output_csv)}")

    metrics_df = pd.DataFrame(fold_metrics)
    mean_row = metrics_df.mean(numeric_only=True)
    mean_row["Fold"] = "Average"
    metrics_df = pd.concat([metrics_df, pd.DataFrame([mean_row])], ignore_index=True)
    summary_path = output_csv.replace(".csv", "_metrics_summary.csv")
    metrics_df.to_csv(summary_path, index=False)
    print(f"Summary saved to: {os.path.abspath(summary_path)}")


def extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script):
    def call_blender(input_fbx, output_npy):
        cmd = [blender_exec, "--background", "--python", blender_script, "--", input_fbx, output_npy]
        subprocess.run(cmd, capture_output=True)

    os.makedirs(npy_root, exist_ok=True)
    for label in ["Autism", "No_Autism"]:
        class_dir = os.path.join(fbx_root, label)
        out_dir = os.path.join(npy_root, label)
        os.makedirs(out_dir, exist_ok=True)
        for fbx_file in os.listdir(class_dir):
            if fbx_file.endswith(".fbx"):
                input_fbx = os.path.join(class_dir, fbx_file)
                output_npy = os.path.join(out_dir, fbx_file.replace(".fbx", ".npy"))
                call_blender(input_fbx, output_npy)

if __name__ == "__main__":
    npy_root = os.path.join(os.getcwd(), "npy_data")
    label_map = {"No_Autism": 0, "Autism": 1}
    train_stgcn(npy_dir=npy_root, label_map=label_map)


Fold 1 Training:   1%|▎                          | 1/75 [00:01<02:13,  1.81s/it]


KeyboardInterrupt: 

In [23]:
# ✅ FBX to ST-GCN Classifier Pipeline (Enhanced with Velocity + Temporal Pooling + Residual Smoothing + Test-Time Augmentation)
import os
import numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import csv
from tqdm import trange

# ✅ Residual Block with Temporal Pooling
class STGCNResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(9,1), stride=(1,1), dropout=0.3):
        super().__init__()
        padding = ((kernel_size[0] - 1) // 2, 0)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) if dropout else nn.Identity()
        self.res_conv = (
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            if in_channels != out_channels or stride != (1,1)
            else nn.Identity()
        )

    def forward(self, x):
        res = self.res_conv(x)
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.dropout(out)
        return self.relu(out + res)

# ✅ Full ST-GCN Model with Temporal Pooling and Residual Skip Connection
class STGCNModel(nn.Module):
    def __init__(self, in_channels=6, num_joints=250, num_classes=2, use_lstm=True):
        super().__init__()
        self.A = torch.eye(num_joints)
        self.block1 = STGCNResidualBlock(in_channels, 64, dropout=0.1)
        self.pool1 = nn.MaxPool2d((2,1))
        self.block2 = STGCNResidualBlock(64, 128, dropout=0.1)
        self.pool2 = nn.AdaptiveMaxPool2d((1, 1))
        self.block3 = STGCNResidualBlock(128, 256, dropout=0.1)
        self.use_lstm = use_lstm

        if self.use_lstm:
            self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
            self.fc = nn.Linear(128, num_classes)
        else:
            self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        A = self.A.to(x.device)
        x = torch.einsum('nctv,vw->nctw', x, A)
        x = self.block1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = self.block3(x)
        x = x.mean(3)

        if self.use_lstm:
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            x = out[:, -1, :]
        else:
            x = x.mean(1)

        return self.fc(x)

# ✅ Dataset with Velocity Only (Stable)
class SkeletonDataset(Dataset):
    def __init__(self, file_paths, labels, augment=False, tta=False):
        self.file_paths = file_paths
        self.labels = labels
        self.augment = augment
        self.tta = tta

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        data = np.load(self.file_paths[idx])
        if data.ndim == 3:
            data = np.transpose(data, (1, 2, 0))

        joints = torch.tensor(data, dtype=torch.float32)
        velocity = joints[:, 1:, :] - joints[:, :-1, :]
        velocity = F.pad(velocity, (0, 0, 0, 1), mode='replicate')
        x = torch.cat([joints, velocity], dim=0)

        if self.augment or self.tta:
            x += torch.randn_like(x) * 0.005

        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y, self.file_paths[idx]

# ✅ Output files will be saved here:
#     → Results saved to: /Users/chancellorwoolsey/Documents/Research Work/images_and_videos/Avatar Dataset

def train_stgcn(npy_dir, label_map, pretrained_ckpt=None, epochs=75, folds=10, output_csv='ResultsFour.csv'):
    fold_metrics = []
    all_paths, all_labels = [], []
    for label, class_id in label_map.items():
        class_dir = os.path.join(npy_dir, label)
        if not os.path.isdir(class_dir):
            continue
        for f in os.listdir(class_dir):
            if f.endswith(".npy"):
                all_paths.append(os.path.join(class_dir, f))
                all_labels.append(class_id)

    paths = np.array(all_paths)
    labels = np.array(all_labels)
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    csv_rows = [["Fold", "VideoPath", "PredictedLabel", "TrueLabel"]]

    for fold, (train_idx, test_idx) in enumerate(skf.split(paths, labels)):
        train_ds = SkeletonDataset(paths[train_idx], labels[train_idx], augment=True)
        test_ds = SkeletonDataset(paths[test_idx], labels[test_idx], augment=False)
        test_ds_tta = SkeletonDataset(paths[test_idx], labels[test_idx], augment=False, tta=True)
        train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
        test_dl = DataLoader(test_ds, batch_size=4)
        tta_dl = DataLoader(test_ds_tta, batch_size=4)

        C, T, V = train_ds[0][0].shape
        model = STGCNModel(in_channels=C, num_joints=V, num_classes=2).to("cpu")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        label_weights = torch.tensor([1.0 for _ in label_map], dtype=torch.float32)
        criterion = nn.CrossEntropyLoss(weight=label_weights)

        best_val_acc = 0.0
        patience, warmup = 10, 10
        epochs_no_improve = 0

        for epoch in trange(epochs, desc=f"Fold {fold+1} Training"):
            model.train()
            for x, y, _ in train_dl:
                x, y = x.to("cpu"), y.to("cpu")
                loss = criterion(model(x), y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            scheduler.step()

            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for x, y, _ in test_dl:
                    preds = torch.argmax(model(x), dim=1)
                    val_correct += (preds == y).sum().item()
                    val_total += y.size(0)
            val_acc = val_correct / val_total
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")
                epochs_no_improve = 0
            elif epoch >= warmup:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    break

        model.load_state_dict(torch.load(f"best_model_fold{fold+1}.pth"))
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for (x_base, y, paths_batch), (x_aug, _, _) in zip(test_dl, tta_dl):
                pred_base = model(x_base)
                pred_aug = model(x_aug)
                pred_final = (pred_base + pred_aug) / 2
                preds = torch.argmax(pred_final, dim=1)
                y_true.extend(y.tolist())
                y_pred.extend(preds.tolist())
                for i in range(len(paths_batch)):
                    csv_rows.append([fold + 1, paths_batch[i], int(preds[i].item()), int(y[i].item())])

        acc = np.mean(np.array(y_true) == np.array(y_pred))
        f1 = f1_score(y_true, y_pred, average='macro')
        prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
        fold_metrics.append({"Fold": fold+1, "Accuracy": acc, "F1": f1, "Precision": prec, "Recall": rec})

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(csv_rows)
    print(f"Results saved to: {os.path.abspath(output_csv)}")

    metrics_df = pd.DataFrame(fold_metrics)
    mean_row = metrics_df.mean(numeric_only=True)
    mean_row["Fold"] = "Average"
    metrics_df = pd.concat([metrics_df, pd.DataFrame([mean_row])], ignore_index=True)
    summary_path = output_csv.replace(".csv", "_metrics_summary.csv")
    metrics_df.to_csv(summary_path, index=False)
    print(f"Summary saved to: {os.path.abspath(summary_path)}")


def extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script):
    def call_blender(input_fbx, output_npy):
        cmd = [blender_exec, "--background", "--python", blender_script, "--", input_fbx, output_npy]
        subprocess.run(cmd, capture_output=True)

    os.makedirs(npy_root, exist_ok=True)
    for label in ["Autism", "No_Autism"]:
        class_dir = os.path.join(fbx_root, label)
        out_dir = os.path.join(npy_root, label)
        os.makedirs(out_dir, exist_ok=True)
        for fbx_file in os.listdir(class_dir):
            if fbx_file.endswith(".fbx"):
                input_fbx = os.path.join(class_dir, fbx_file)
                output_npy = os.path.join(out_dir, fbx_file.replace(".fbx", ".npy"))
                call_blender(input_fbx, output_npy)

if __name__ == "__main__":
    npy_root = os.path.join(os.getcwd(), "npy_data")
    label_map = {"No_Autism": 0, "Autism": 1}
    train_stgcn(npy_dir=npy_root, label_map=label_map)


Fold 1 Training:   8%|██▏                        | 6/75 [00:07<01:26,  1.26s/it]


KeyboardInterrupt: 

In [21]:
# ✅ FBX to ST-GCN Classifier Pipeline (High-Resolution Input, No Convs, Learnable Graph)
import os
import numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import csv
from tqdm import trange

# ✅ Simplified Block Without Convolutions
class STGCNSkipBlock(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x

# ✅ Full ST-GCN Model with Learnable Graph (No Conv)
class STGCNModel(nn.Module):
    def __init__(self, in_channels=6, num_joints=250, num_classes=2, use_lstm=True):
        super().__init__()
        self.A = nn.Parameter(torch.eye(num_joints))
        self.skip1 = STGCNSkipBlock()
        self.skip2 = STGCNSkipBlock()
        self.skip3 = STGCNSkipBlock()
        self.use_lstm = use_lstm

        if self.use_lstm:
            self.lstm = nn.LSTM(input_size=in_channels, hidden_size=128, batch_first=True)
            self.fc = nn.Linear(128, num_classes)
        else:
            self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        A = F.softmax(self.A, dim=-1).to(x.device)
        x = torch.einsum('nctv,vw->nctw', x, A)
        x = self.skip1(x)
        x = self.skip2(x)
        x = self.skip3(x)
        x = x.mean(3)

        if self.use_lstm:
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            x = out[:, -1, :]
        else:
            x = x.mean(1)

        return self.fc(x)

# ✅ Dataset with Velocity
class SkeletonDataset(Dataset):
    def __init__(self, file_paths, labels, augment=False, tta=False):
        self.file_paths = file_paths
        self.labels = labels
        self.augment = augment
        self.tta = tta

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        data = np.load(self.file_paths[idx])
        if data.ndim == 3:
            data = np.transpose(data, (1, 2, 0))

        joints = torch.tensor(data, dtype=torch.float32)
        velocity = joints[:, 1:, :] - joints[:, :-1, :]
        velocity = F.pad(velocity, (0, 0, 0, 1), mode='replicate')
        x = torch.cat([joints, velocity], dim=0)

        if self.augment or self.tta:
            x += torch.randn_like(x) * 0.005

        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y, self.file_paths[idx]

# ✅ Training Pipeline

def train_stgcn(npy_dir, label_map, output_csv='ResultsFive.csv', epochs=75, folds=10):
    fold_metrics, all_paths, all_labels = [], [], []
    for label, class_id in label_map.items():
        label_path = os.path.join(npy_dir, label)
        for f in os.listdir(label_path):
            if f.endswith(".npy"):
                all_paths.append(os.path.join(label_path, f))
                all_labels.append(class_id)

    paths = np.array(all_paths)
    labels = np.array(all_labels)
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    csv_rows = [["Fold", "VideoPath", "PredictedLabel", "TrueLabel"]]

    for fold, (train_idx, test_idx) in enumerate(skf.split(paths, labels)):
        print(f"\n=== Fold {fold+1}/{folds} ===")
        train_ds = SkeletonDataset(paths[train_idx], labels[train_idx], augment=True)
        test_ds = SkeletonDataset(paths[test_idx], labels[test_idx], augment=False, tta=True)
        train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
        test_dl = DataLoader(test_ds, batch_size=4)

        C, T, V = train_ds[0][0].shape
        model = STGCNModel(in_channels=C, num_joints=V, num_classes=2).to("cpu")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        weights = torch.tensor([1.0, 1.0])
        criterion = nn.CrossEntropyLoss(weight=weights)

        best_val_acc, patience, warmup = 0.0, 10, 10
        epochs_no_improve = 0

        for epoch in trange(epochs, desc=f"Fold {fold+1} Training"):
            model.train()
            for x, y, _ in train_dl:
                optimizer.zero_grad()
                loss = criterion(model(x), y)
                loss.backward()
                optimizer.step()
            scheduler.step()

            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for x, y, _ in test_dl:
                    pred = torch.argmax(model(x), dim=1)
                    val_correct += (pred == y).sum().item()
                    val_total += y.size(0)
            val_acc = val_correct / val_total
            print(f"  Epoch {epoch+1} Val Acc: {val_acc:.4f}")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")
                epochs_no_improve = 0
            elif epoch > warmup:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    break

        model.load_state_dict(torch.load(f"best_model_fold{fold+1}.pth"))
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for x, y, paths_batch in test_dl:
                out = model(x)
                preds = torch.argmax(out, dim=1)
                y_true.extend(y.tolist())
                y_pred.extend(preds.tolist())
                for i in range(len(paths_batch)):
                    csv_rows.append([fold + 1, paths_batch[i], int(preds[i].item()), int(y[i].item())])

        acc = np.mean(np.array(y_true) == np.array(y_pred))
        f1 = f1_score(y_true, y_pred, average='macro')
        prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
        fold_metrics.append({"Fold": fold+1, "Accuracy": acc, "F1": f1, "Precision": prec, "Recall": rec})

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(csv_rows)
    print(f"✅ Results saved to: {os.path.abspath(output_csv)}")

    metrics_df = pd.DataFrame(fold_metrics)
    mean_row = metrics_df.mean(numeric_only=True)
    mean_row["Fold"] = "Average"
    metrics_df = pd.concat([metrics_df, pd.DataFrame([mean_row])], ignore_index=True)
    summary_path = output_csv.replace(".csv", "_metrics_summary.csv")
    metrics_df.to_csv(summary_path, index=False)
    print(f"📊 Summary saved to: {os.path.abspath(summary_path)}")

# ✅ FBX Extraction

def extract_fbx_to_npy(fbx_root, npy_root, blender_exec, blender_script):
    def call_blender(input_fbx, output_npy):
        cmd = [blender_exec, "--background", "--python", blender_script, "--", input_fbx, output_npy]
        subprocess.run(cmd, capture_output=True)

    os.makedirs(npy_root, exist_ok=True)
    for label in ["Autism", "No_Autism"]:
        class_dir = os.path.join(fbx_root, label)
        out_dir = os.path.join(npy_root, label)
        os.makedirs(out_dir, exist_ok=True)
        for fbx_file in os.listdir(class_dir):
            if fbx_file.endswith(".fbx"):
                input_fbx = os.path.join(class_dir, fbx_file)
                output_npy = os.path.join(out_dir, fbx_file.replace(".fbx", ".npy"))
                call_blender(input_fbx, output_npy)

# ✅ Entrypoint
if __name__ == "__main__":
    npy_root = os.path.join(os.getcwd(), "npy_data")
    label_map = {"No_Autism": 0, "Autism": 1}
    train_stgcn(npy_dir=npy_root, label_map=label_map)



=== Fold 1/10 ===


Fold 1 Training:   1%|▎                          | 1/75 [00:00<00:44,  1.67it/s]

  Epoch 1 Val Acc: 0.4333


Fold 1 Training:   3%|▋                          | 2/75 [00:01<00:43,  1.66it/s]

  Epoch 2 Val Acc: 0.5667


...

  Epoch 4 Val Acc: 0.6000


Fold 1 Training:   7%|█▊                         | 5/75 [00:02<00:40,  1.74it/s]

  Epoch 5 Val Acc: 0.5667


Fold 1 Training:   7%|█▊                         | 5/75 [00:03<00:48,  1.44it/s]


KeyboardInterrupt: 

## Ensemble Results

In [ ]:
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# === 📂 SET YOUR INPUT FOLDER PATH ===
input_folder = '/results'

# === 📥 LOAD ALL CSV FILES WITH REQUIRED COLUMNS ===
dfs = []
filenames = []
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        path = os.path.join(input_folder, file)
        df = pd.read_csv(path)
        if 'TrueLabel' in df.columns and 'PredictedLabel' in df.columns:
            dfs.append(df)
            filenames.append(file)

# === ✅ VALIDATION CHECK ===
if len(dfs) < 2:
    raise ValueError("❌ Need at least 2 CSV files with 'TrueLabel' and 'PredictedLabel' columns.")

# === 🧠 TRUE LABELS (assumed same across all files) ===
true_labels = dfs[0]['TrueLabel'].values

# === 📈 METRIC FUNCTION ===
def compute_metrics(true, pred):
    return {
        "autism (1)": {
            "precision": precision_score(true, pred, pos_label=1),
            "recall": recall_score(true, pred, pos_label=1),
            "f1": f1_score(true, pred, pos_label=1)
        },
        "no autism (0)": {
            "precision": precision_score(true, pred, pos_label=0),
            "recall": recall_score(true, pred, pos_label=0),
            "f1": f1_score(true, pred, pos_label=0)
        },
        "overall": {
            "accuracy": accuracy_score(true, pred)
        }
    }

# === 📄 INDIVIDUAL MODEL METRICS ===
print("=== 📊 Individual Model Results ===")
for fname, df in zip(filenames, dfs):
    print(f"\n📂 File: {fname}")
    metrics = compute_metrics(true_labels, df['PredictedLabel'])
    for cls, scores in metrics.items():
        print(f"Class: {cls}")
        for k, v in scores.items():
            print(f"  {k}: {v:.4f}")

# === 🗳️ ENSEMBLE: MAJORITY VOTE ACROSS MODELS ===
all_preds = pd.DataFrame({f"model_{i}": df["PredictedLabel"] for i, df in enumerate(dfs)})
ensemble_preds = all_preds.mode(axis=1)[0]  # majority vote

# === 🧠 ENSEMBLE METRICS ===
print("\n\n=== 🧠 Ensemble (Majority Vote) Results ===")
ensemble_metrics = compute_metrics(true_labels, ensemble_preds)
for cls, scores in ensemble_metrics.items():
    print(f"Class: {cls}")
    for k, v in scores.items():
        print(f"  {k}: {v:.4f}")
